In [7]:
using Dojo
using DojoEnvironments
using LinearAlgebra

# ### Environment
quadrotor_env = get_environment(:quadrotor_waypoint; horizon=1000)

# ### Controllers
trans_mode = normalize([1;1;1;1])

function distribute_thrust(thrust, torque)
    # Calculate individual thrusts based on desired thrust and torque
    T1 = thrust[3] + torque[1] + torque[2] - torque[3]
    T2 = thrust[3] - torque[1] + torque[2] + torque[3]
    T3 = thrust[3] - torque[1] - torque[2] - torque[3]
    T4 = thrust[3] + torque[1] - torque[2] + torque[3]
    return [T1, T2, T3, T4]
end

function velocity_controller!(environment, v_des)33
    state = get_state(environment)
    position = state[1:3] # x, y, z
    orientation = state[4:6] # axis*angle
    linear_velocity = state[7:9] # vx, vy, vz
    angular_velocity = state[10:12] # ωx, ωy, ωz 

    error_v = v_des .- linear_velocity
    thrust = (10 .* error_v .- 1 .* linear_velocity .+ 5.1)
    println("Thrust: ", thrust)
    
    # Calculate torque (simplified for demonstration purposes)
    # Assume that we want to stabilize the orientation (no rotation)
    torque = -1 .* angular_velocity # D term for angular velocity
    println("Torque: ", torque)

    rpm_thrust = distribute_thrust(thrust, torque) 
    println("RPM thrust: ", rpm_thrust)
    rpm = rpm_thrust .* 20
    println("RPM: ", rpm)
    set_input!(environment, rpm)
end

function position_controller!(environment, pos_des)
    pos_is = get_state(environment)[1:3] # x, y, z
    v_des = pos_des .- pos_is
    velocity_controller!(environment, v_des)
end

function controller!(environment, k)
    # Desired position (x, y, z)
    pos_des = [1;1;0.3] # Example coordinates to fly to
    position_controller!(environment, pos_des)
end

# ### Simulate
initialize!(quadrotor_env, :quadrotor)
simulate!(quadrotor_env, controller!; record=true)

# ### Visualize
vis = visualize(quadrotor_env)
render(vis)


Thrust: [15.1, 15.1, 7.249999999999999]
Torque: [-0.0, -0.0, -0.0]
RPM thrust: [7.249999999999999, 7.249999999999999, 7.249999999999999, 7.249999999999999]
RPM: [144.99999999999997, 144.99999999999997, 144.99999999999997, 144.99999999999997]
Thrust: [15.099999999999998, 15.1, -0.7715214085381259]
Torque: [-0.0, -4.444940107437749e-16, -0.0]
RPM thrust: [-0.7715214085381263, -0.7715214085381263, -0.7715214085381255, -0.7715214085381255]
RPM: [-15.430428170762527, -15.430428170762527, -15.430428170762509, -15.430428170762509]
Thrust: [15.099999999999998, 15.1, 0.31487152590057477]
Torque: [1.1017491399210138e-16, -5.740935306302243e-16, 7.59394373699248e-34]
RPM thrust: [0.3148715259005743, 0.3148715259005741, 0.3148715259005752, 0.31487152590057543]
RPM: [6.2974305180114865, 6.297430518011482, 6.297430518011504, 6.297430518011509]
Thrust: [15.099999999999998, 15.1, 1.3075543750223848]
Torque: [1.5840027080382138e-16, -6.19880012766643e-16, 2.3967521031495216e-33]
RPM thrust: [1.30755437

SingularException: SingularException(6)